## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)
study_results
# Combine the data into a single dataset
combined_df= mouse_metadata.merge(study_results, on='Mouse ID', how='inner')

# Display the data table for preview
combined_df.sample(10)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
1645,v835,Naftisol,Male,7,29,20,51.296543,2
354,g316,Capomulin,Female,22,22,15,45.216743,0
1500,s166,Placebo,Male,19,27,5,47.186788,0
616,b759,Ceftamin,Female,12,25,25,55.083632,0
1549,t451,Stelasyn,Male,8,29,40,67.654796,1
328,j119,Capomulin,Female,7,23,0,45.000000,0
1847,y865,Ceftamin,Male,23,26,5,46.735006,1
396,e662,Ramicane,Male,8,24,25,44.664594,1
1004,j755,Naftisol,Male,23,27,0,45.000000,0
406,u196,Ramicane,Male,18,25,25,46.705325,1


In [2]:
# Checking the number of mice.
num_cases = len(combined_df)
print(f'Number of Cases = {num_cases}')

num_mice = len(combined_df['Mouse ID'].unique())
print(f'Number of Mice = {num_mice}')

Number of Cases = 1893
Number of Mice = 249


In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dupe_mouse_mask = combined_df.duplicated(subset=['Mouse ID'])

dupe_mice = combined_df[dupe_mouse_mask].sample(100)
dupe_timepoint = dupe_mice.duplicated(subset=['Timepoint'])
mice_duplicates = pd.DataFrame({'Mouse ID': dupe_mice[dupe_timepoint]['Mouse ID']})

print(mice_duplicates)

#clean_df = combined_df.query("Mouse ID in mice_duplicates")


     Mouse ID
1464     r604
749      c927
1272     o331
662      c302
91       j913
...       ...
480      a262
1121     l558
1800     x930
166      a644
301      b128

[91 rows x 1 columns]


In [4]:
# Optional: Get all the data for the duplicate mouse ID. 
unclean_df = combined_df[combined_df.isin(mice_duplicates)]
#unclean_df.to_excel('optional_unclean.xlsx', engine='xlsxwriter') 

In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = combined_df[~combined_df.isin(mice_duplicates)].dropna()
#clean_df.to_excel('output1.xlsx', engine='xlsxwriter') 

In [6]:
# Checking the number of mice in the clean DataFrame.
count_clean_df = len(clean_df['Mouse ID'].unique())
print(f'Number of Mice that have no Timepoint duplication: {count_clean_df}')

Number of Mice that have no Timepoint duplication: 249


## Summary Statistics

In [18]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
grouped_clean = clean_df.groupby(by=['Drug Regimen'])
clean_df_mean = clean_df.groupby(by=['Drug Regimen']).mean()
clean_df_median = clean_df.groupby(by=['Drug Regimen']).median()
clean_df_var = clean_df.groupby(by=['Drug Regimen']).var()
clean_df_stdev = clean_df.groupby(by=['Drug Regimen']).std()
clean_df_sem = clean_df.groupby(by=['Drug Regimen']).sem()

summary_stats_df = pd.DataFrame({'TumorVol Mean':clean_df_mean['Tumor Volume (mm3)'],
                                'TumorVol Median':clean_df_median['Tumor Volume (mm3)'],
                                'TumorVol Variance':clean_df_var['Tumor Volume (mm3)'],
                                 'TumorVol StdDev':clean_df_stdev['Tumor Volume (mm3)'],
                                 'TumorVol StdErr':clean_df_sem['Tumor Volume (mm3)']})


summary_stats_df

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



,TumorVol Mean,TumorVol Median,TumorVol Variance,TumorVol StdDev,TumorVol StdErr
Drug Regimen,,,,,
Capomulin,40.654440,41.557809,25.442762,5.044082,0.340072
Ceftamin,52.526669,51.635527,38.868834,6.234487,0.478163
Infubinol,52.769281,51.736561,43.760370,6.615162,0.508859
Ketapril,55.343296,53.757945,69.695983,8.348412,0.623990
Naftisol,54.314902,52.484804,67.568073,8.219980,0.617852
Placebo,53.778949,51.950574,60.647436,7.787646,0.592084
Propriva,52.144882,50.137432,42.753707,6.538632,0.530353
Ramicane,40.133035,40.667713,23.702562,4.868528,0.330497
Stelasyn,53.995205,52.154203,58.212757,7.629728,0.588646


In [12]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
df_grouped = clean_df.groupby(by='Drug Regimen')

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
